## uproot overview

Uproot is a pure Python + Numpy reader of ROOT files.

   * Without a C++ layer, there are no memory ownership issues between C++ and Python.
   * Different design: instead of delivering event objects, uproot delivers columns of data as (jagged) arrays.
   * Not hampered by slow Python execution because data in ROOT files are laid out as (jagged) arrays: just need to cast them as Numpy arrays.

_(Disclosure: I'm the author of uproot.)_

In uproot, files, directories within files, and TTrees/TBranches behave like Python dicts.

In [ ]:
import uproot
file = uproot.open("http://scikit-hep.org/uproot/examples/Event.root")
file.keys()

In [ ]:
file["ProcessID0"]

In [ ]:
file["htime"]

In [ ]:
tree = file["T"]
tree

In [ ]:
tree.keys()   # allkeys()

To get a sense of what a TTree contains, use `show`.

In [ ]:
tree.show()

To read a (jagged) array, call `array` or `arrays`.

In [ ]:
tree["fTracks.fMass2"].array()

In [ ]:
tree.array("fTracks.fMass2")

In [ ]:
tree.arrays(["fTracks.fMass2", "fTracks.fCharge"])

## Interpretations

The translation from ROOT data to an array is given by the branch's `interpretation` (if it has one).

In [ ]:
tree["fNtrack"].interpretation

In [ ]:
tree["fTemperature"].interpretation

In [ ]:
tree["fMatrix[4][4]"].interpretation

In [ ]:
tree["fTracks.fMass2"].interpretation

In [ ]:
tree["fTracks.fCharge"].interpretation

In [ ]:
tree["fH"].interpretation

If a branch has no `interpretation`, it can't be read. Either it's a no-data branch (exists just for structure) or it's an instance of uproot's incompleteness.

In [ ]:
print(tree["fTracks.fPointValue"].interpretation)   # as of April 2019, this one has no interpretation

The bytes can be read and even divided along entry boundaries, but we don't yet know how to turn the bytes into an array.

In [ ]:
uproot.asdebug

In [ ]:
tree["fTracks.fPointValue"].array(uproot.asdebug)

Complex classes are generated based on the ROOT file's self-describing streamers, but they aren't necessarily fast to read (more Python than Numpy).

In [ ]:
tree["fH"].interpretation

In [ ]:
histograms = tree["fH"].array()
histograms

In [ ]:
histograms[0].__dict__

## Fitting into memory constraints

Restricting the range of entries avoids reading too many baskets (chunks on disk).

In [ ]:
tree.numentries

In [ ]:
tree["fMatrix[4][4]"].numbaskets

In [ ]:
tree["fMatrix[4][4]"].array(entrystart=600, entrystop=800)

Typically, you'd want to read chunk of entries from all interesting branches, do some work, then move on to the next chunk: use `iterate`.

In [ ]:
import numpy
for arrays in tree.iterate(["fTracks.fPx", "fTracks.fPy"], entrysteps=300):
    mag = numpy.sqrt(arrays[b"fTracks.fPx"]**2 + arrays[b"fTracks.fPy"]**2)
    print(len(mag), mag[0][0])

The same for a set of files is `uproot.iterate` (supply file names with wildcards and tree name).

In [ ]:
# no wildcards for XRootD and HTTP
filenames = ["http://scikit-hep.org/uproot/examples/HZZ" + x + ".root" for x in ["", "-zlib", "-lz4", "-lzma"]]
for arrays in uproot.iterate(filenames, "events", ["Muon_Px", "Muon_Py"]):
    mag = numpy.sqrt(arrays[b"Muon_Px"]**2 + arrays[b"Muon_Py"]**2)
    print(len(mag), mag[1][0])

## Caching

If you 

## Parallel processing

## Lazy evaluation

## Dask (distributed computing)